# Create secrets.yaml

Expected fields (no quotes):

```yaml
client_id: 123
client_secret: abc
client_key: abc
```

# Using the interface:

## Construct an API query

A query is a combination of a "type (and, or, not)" with multiple Conditions ("Cond")

Each condition is a combination of a "field" (Fields, F), "value" and a operation ("Operations", "Op").

In [2]:
from tiktok_research_api_helper.query import VideoQuery, Cond, Fields, Op

query = VideoQuery(
        and_=[
            Cond(Fields.hashtag_name, "garfield", Op.EQ),
            Cond(Fields.region_code, "US", Op.EQ),

            # Alternative version with multiple countries - Then the operation changes to "IN" instead of "EQ" (equals) as it's a list
            # the library handles list vs str natively
            # Cond(Fields.region_code, ["US", "UK"], Op.IN),
        ],
    )

## TikTokApiClient provides a high-level interface to fetch all api results, and optionally store them in a database

In [3]:
from pathlib import Path
from datetime import datetime
from tiktok_research_api_helper.api_client import ApiClientConfig, TikTokApiClient, VideoQueryConfig

config = ApiClientConfig(engine=None,
                         api_credentials_file=Path("./secrets.yaml"))
api_client = TikTokApiClient.from_config(config)

query_config = VideoQueryConfig(query=query,
                                start_date=datetime.fromisoformat("2024-03-03"),
                                end_date=datetime.fromisoformat("2024-03-04"),
                                fetch_user_info=True,
                                fetch_comments=False)

### `api_results_iter` yields each API reponse as a parsed `TikTokApiClientFetchResult`. Iteration stops when the API indicates the query results have been fully delivered

In [4]:
[result for result in api_client.api_results_iter(query_config)]

[TikTokApiClientFetchResult(videos=[{'username': 'christina.patricio', 'view_count': 642, 'create_time': 1709596108, 'hashtag_names': ['movie', 'movies', 'garfield', 'fyp', 'animationmovie', 'contentcreator', 'thegarfieldshow', 'thepatricios', 'garfieldmovie', 'foryoupage', 'garfieldandfriends', 'thegarfieldmovie'], 'id': 7342659269533142315, 'region_code': 'US', 'video_description': '#fyp #foryoupage #thepatricios #contentcreator #movie #movies #garfield #thegarfieldshow #thegarfieldmovie #garfieldandfriends #garfieldmovie #animationmovie @Sony Pictures ', 'comment_count': 0, 'like_count': 32, 'music_id': 7342659412659604267, 'share_count': 0}, {'id': 7342658281372011818, 'region_code': 'US', 'username': '4amnetwork', 'video_description': 'I am shocked when a movie trailer is less than 2 minutes long these days. I assume this needs all the marketing it can get though. #movietok #foryou #movietrailer #fyp #filmtok #fypシ #fypp #garfield ', 'comment_count': 4, 'create_time': 1709595891, 

### `fetch_all` fetches all API results and returns a single TikTokApiClientFetchResult with all API results.
api_client.fetch_all(query_config)

In [4]:
# If you provide a SqlAlchemy engine in the ApiClientConfig you can use TikTokApiClient to store results as they are received
api_client.fetch_all(store_results_after_each_response=True) # or equivalent function fetch_and_store_all()

### You can limit the number of requests the client makes to the API (reduce quota usage)

In [4]:
config = ApiClientConfig(max_api_requests=1,
                         api_credentials_file=Path("./secrets.yaml"))
api_client = TikTokApiClient.from_config(config)

print("max_api_requests_reached:", api_client.max_api_requests_reached)
# now client will stop after making 1 request. NOTE: to make more requests once max_api_requests has been reached you will need to call client.reset_num_requests()
print("num results from api_results_iter:", len([result for result in api_client.api_results_iter(query_config)]))
print("max_api_requests_reached:", api_client.max_api_requests_reached)
print("num results from api_results_iter:", len([result for result in api_client.api_results_iter(query_config)]))
print("Reseting num requests")
api_client.reset_num_requests()
print("max_api_requests_reached:", api_client.max_api_requests_reached)

max_api_requests_reached: False
num results from api_results_iter: 1
max_api_requests_reached: True
num results from api_results_iter: 0
Reseting num requests
max_api_requests_reached: False


## TikTokApiRequestClient and TikTokRequest provide a lower-level interface to API

In [7]:
from pathlib import Path
from tiktok_research_api_helper.api_client import TikTokApiRequestClient, TikTokVideoRequest

# reads from secrets.yaml in the same directory
request_client = TikTokApiRequestClient.from_credentials_file(Path("./secrets.yaml"))

In [8]:
from tiktok_research_api_helper.query import VideoQuery, Cond, Fields, Op
# sample query
req = TikTokVideoRequest(
    query=VideoQuery(or_=Cond(Fields.video_id, ["7345557461438385450", "123456"], Op.IN)),
    start_date="20240301",
    end_date="20240329",
)

req

TikTokVideoRequest(query=VideoQuery(and_=None, or_=[Condition(field=_Field(name='video_id', validator=<instance_of validator for type <class 'str'>>), field_values=['7345557461438385450', '123456'], operation=<Operations.IN: 'IN'>)], not_=None), start_date='20240301', end_date='20240329', max_count=100, is_random=False, cursor=None, search_id=None)

In [9]:
request_client.fetch_videos(req)

TikTokVideoResponse(data={'cursor': 1, 'has_more': False, 'search_id': '7390522638017451050', 'videos': [{'region_code': 'US', 'share_count': 7603, 'video_description': 'Friends who nap together>>>>>> Watch the Peanuts Classics on Apple TV+! ', 'comment_count': 662, 'create_time': 1710270898, 'hashtag_names': [], 'id': 7345557461438385450, 'music_id': 7345557575716440875, 'like_count': 131020, 'username': 'snoopy', 'view_count': 673184}]}, error={'code': 'ok', 'message': '', 'log_id': '202407120009576BF37F35AE2817047133'}, videos=[{'region_code': 'US', 'share_count': 7603, 'video_description': 'Friends who nap together>>>>>> Watch the Peanuts Classics on Apple TV+! ', 'comment_count': 662, 'create_time': 1710270898, 'hashtag_names': [], 'id': 7345557461438385450, 'music_id': 7345557575716440875, 'like_count': 131020, 'username': 'snoopy', 'view_count': 673184}])

In [ ]:
request_client = TikTokApiRequestClient.from_credentials_file(Path("./secrets.yaml"),
                                                              max_api_requests=4) # This is optional, included here to limit output
hashtags = ["fyp", "foryou", "foryoupage"]

# Multiple conditions
req = TikTokVideoRequest(
    query=VideoQuery(
        and_=[
            Cond(Fields.hashtag_name, hashtags, Op.IN),
            Cond(Fields.region_code, "US", Op.EQ),

            # Alternative version with multiple countries - Then the operation changes to "IN" instead of "EQ" (equals) as it's a list
            # the library handles list vs str natively
            # Cond(Fields.region_code, ["US", "UK"], Op.IN),

        ]
    ),
    start_date="20220101",
    end_date="20220129",
)

request_client.fetch_videos(req)